# Python内存管理及释放

python对内存释放并不及时，例如a=range(10000*10000)，会发现内存飙升一个多G，del a 或者a=[]都不能将内存降下来，希望马上释放可以：

In [1]:
a = range(10000)
import gc # garbage collector
del a 
gc.collect()

57

如果你是指在自己的程序中想删除所有全局变量的话，可以自己编写一个clear函数，通过globals()获取全局变量然后将其中不需要的内容删除，例如下面的程序保留函数，类，模块，删除所有其它全局变量：

In [2]:
def clear():
    for key, value in globals().items():
        if callable(value) or value.__class__.__name__ == "module":
            continue
        del globals()[key]

不过程序中应该避免这种对全局变量的依赖。

先从较浅的层面来说，Python的内存管理机制可以从三个方面来讲

（1）垃圾回收

（2）引用计数

（3）内存池机制

一、垃圾回收：

python不像C++，Java等语言一样，他们可以不用事先声明变量类型而直接对变量进行赋值。对Python语言来讲，对象的类型和内存都是在运行时确定的。这也是为什么我们称Python语言为动态类型的原因（这里我们把动态类型可以简单的归结为对变量内存地址的分配是在运行时自动判断变量类型并对变量进行赋值）。

当内存中有不再使用的部分时，垃圾收集器就会把他们清理掉。它会去检查那些引用计数为0的对象，然后清除其在内存的空间。当然除了引用计数为0的会被清除，还有一种情况也会被垃圾收集器清掉：当两个对象相互引用时，他们本身其他的引用已经为0了. 垃圾回收机制还有一个循环垃圾回收器, 确保释放循环引用对象(a引用b, b引用a, 导致其引用计数永远不为0)。

Python同时采用了分代(generation)回收的策略。这一策略的基本假设是，存活时间越久的对象，越不可能在后面的程序中变成垃圾。我们的程序往往会产生大量的对象，许多对象很快产生和消失，但也有一些对象长期被使用。出于信任和效率，对于这样一些“长寿”对象，我们相信它们的用处，所以减少在垃圾回收中扫描它们的频率。

Python将所有的对象分为0，1，2三代。所有的新建对象都是0代对象。当某一代对象经历过垃圾回收，依然存活，那么它就被归入下一代对象。垃圾回收启动时，一定会扫描所有的0代对象。如果0代经过一定次数垃圾回收，那么就启动对0代和1代的扫描清理。当1代也经历了一定次数的垃圾回收后，那么会启动对0，1，2，即对所有对象进行扫描。

这两个次数即上面get_threshold()返回的(700, 10, 10)返回的两个10。也就是说，每10次0代垃圾回收，会配合1次1代的垃圾回收；而每10次1代的垃圾回收，才会有1次的2代垃圾回收。

同样可以用set_threshold()来调整，比如对2代对象进行更频繁的扫描。gc.set_threshold(700, 10, 5)

二、引用计数：

Python采用了类似Windows内核对象一样的方式来对内存进行管理。每一个对象，都维护这一个对指向该对对象的引用的计数。例如 x=3.14 y=x 我们首先创建了一个对象3.14， 然后将这个浮点数对象的引用赋值给x，因为x是第一个引用，因此，这个浮点数对象的引用计数为1. 语句y = x创建了一个指向同一个对象的引用别名y，我们发现，并没有为Y创建一个新的对象，而是将Y也指向了x指向的浮点数对象，使其引用计数为2. 对于C语言来讲，我们创建一个变量A时就会为为该变量申请一个内存空间，并将变量值 放入该空间中,当将该变量赋给另一变量B时会为B申请一个新的内存空间，并将变量值放入到B的内存空间中，这也是为什么A和B的指针不一致的原因。

三、内存池机制

Python的内存机制以金字塔行，-1，-2层主要有操作系统进行操作，

　　第0层是C中的malloc，free等内存分配和释放函数进行操作；

　　第1层和第2层是内存池，有Python的接口函数PyMem_Malloc函数实现，当对象小于256K时有该层直接分配内存；

　　第3层是最上层，也就是我们对Python对象的直接操作；

在 C 中如果频繁的调用 malloc 与 free 时,是会产生性能问题的.再加上频繁的分配与释放小块的内存会产生内存碎片. Python 在这里主要干的工作有:

　　如果请求分配的内存在1~256字节之间就使用自己的内存管理系统,否则直接使用 malloc.

　　这里还是会调用 malloc 分配内存,但每次会分配一块大小为256k的大块内存.

　　经由内存池登记的内存到最后还是会回收到内存池,并不会调用 C 的 free 释放掉.以便下次使用.对于简单的Python对象，例如数值、字符串，元组（tuple不允许被更改)采用的是复制的方式(深拷贝?)，也就是说当将另一个变量B赋值给变量A时，虽然A和B的内存空间仍然相同，但当A的值发生变化时，会重新给A分配空间，A和B的地址变得不再相同. 而对于像字典(dict)，列表(List)等，改变一个就会引起另一个的改变，也称之为浅拷贝. 
  
  附：

引用计数增加

1.对象被创建：x=4

2.另外的别人被创建：y=x

3.被作为参数传递给函数：foo(x)

4.作为容器对象的一个元素：a=[1,x,'33']

引用计数减少

1.一个本地引用离开了它的作用域。比如上面的foo(x)函数结束时，x指向的对象引用减1。

2.对象的别名被显式的销毁：del x ；或者del y

3.对象的一个别名被赋值给其他对象：x=789

4.对象从一个窗口对象中移除：myList.remove(x)

5.窗口对象本身被销毁：del myList，或者窗口对象本身离开了作用域。

In [7]:
A = (1, 2, 'hello')
B = A
print(id(A))
print(id(B))
A = (1, 2)
print(id(A))
print(id(B))
print(B)

C = {'a':'hello', 'b':'world','c':100}
D = C
print(id(C))
print(id(D))
C['a'] = 'real'
print(id(C))
print(id(D))
print(D)

1434411984072
1434411984072
1434410481160
1434411984072
(1, 2, 'hello')
1434412211080
1434412211080
1434412211080
1434412211080
{'a': 'real', 'b': 'world', 'c': 100}


在Python中，整数和短小的字符，Python都会缓存这些对象，以便重复使用。当我们创建多个等于1的引用时，实际上是让所有这些引用指向同一个对象。比如，所有整数1的引用都指向同一对象。即使使用赋值语句，也只是创造了新的引用，而不是对象本身。长的字符串和其它对象可以有多个相同的对象，可以使用赋值语句创建出新的对象。为了检验两个引用指向同一个对象，我们可以用is关键字。is用于判断两个引用所指的对象是否相同。

In [8]:
a = 1
b = 1
print(id(a))
print(id(b))
print(a is b)
a = []
b = []
print(a is b)
a = 'very good, jim'
b = 'very good, jim'
print(a is b)

140733334000448
140733334000448
True
False
False


在Python中，每个对象都有存有指向该对象的引用总数，即引用计数(reference count)。

我们可以使用sys包中的getrefcount()，来查看某个对象的引用计数。需要注意的是，当使用某个引用作为参数，传递给getrefcount()时，参数实际上创建了一个临时的引用。因此，getrefcount()所得到的结果，会比期望的多1。

In [11]:
from sys import getrefcount
a = [1, 2]
print(getrefcount(a))
b = a
print(getrefcount(a))
c = [a, a]
print(getrefcount(a))

2
3
5


对象引用对象
Python的一个容器对象(container)，比如表、词典等，可以包含多个对象。实际上，容器对象中包含的并不是元素对象本身，是指向各个元素对象的引用。

我们也可以自定义一个对象，并引用其它对象:

In [12]:
class from_obj(object):
    def __init__(self, to_obj):
        self.to_obj = to_obj

b = [1,2,3]
a = from_obj(b)
print(id(a.to_obj))
print(id(b))

1434410763720
1434410763720
